# BMTC

> Bengaluru Metropolitan Transport Corporation (BMTC) is a state-owned public road transport corporation in the Indian city of Bengaluru.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
from pathlib import Path
import datetime
from tqdm import tqdm
import geojson

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

import requests
import pandas as pd

from traffic_data_bengaluru.utils import *

In [ ]:
#| hide
#| eval: false

data_directory = Path('../data/bmtc/')
data_directory.mkdir(exist_ok=True, parents=True)

# Routes

> Takes a route text for regex matching and returns matching routes in response.

**Sample request**

```bash
curl 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchRoute_v2' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  -H 'lan: en' \
  --data-raw '{"routetext":"210"}'
```

**Sample response**

```json
{
    "data": [
        {
            "union_rowno": 2,
            "row": 1,
            "routeno": "210-A",
            "responsecode": 200,
            "routeparentid": 7426
        },
        {
            "union_rowno": 2,
            "row": 3,
            "routeno": "210-AA",
            "responsecode": 200,
            "routeparentid": 7427
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 53,
    "responsecode": 200
}
```

In [ ]:
#| export
def fetch_routes(pattern: str = "", sleep_duration: float = 0.1):
    """Fetches BMTC routes matching the given pattern, or all routes if pattern is empty."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchRoute_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if pattern != '':
        payload = {"routetext": pattern}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        routes = []
        characters = string.digits + string.ascii_lowercase
        for pattern in tqdm(characters, desc = 'Fetching routes'):
            routes += fetch_routes(pattern)
        return routes

In [ ]:
#| hide
#| eval: false

routes = fetch_routes(pattern = '210-NA')
print(json.dumps(routes, indent=4))

In [ ]:
#| export
def process_routes(routes):
    """Process and clean route data, returning a DataFrame with `route_id` and `route_number`."""
    df_routes = pd.DataFrame(routes)
    df_routes = df_routes.drop_duplicates(subset=["routeno"], keep="first")

    df_routes.rename(columns = {'routeno': 'route_number', 'routeparentid': 'route_id'}, inplace=True)
    df_routes = df_routes.sort_values(by='route_id').reset_index(drop=True)
    columns = ['route_id', 'route_number']
    return df_routes[columns]

In [ ]:
#| hide
#| eval: false
df_routes = process_routes(routes)
df_routes

# Route Points

> Takes `route_id` as input and returns geo-locations of the route to be taken in response. There isn't an identifier for the bus stop in this dataset. Instead it'points are at regular distance along the route.

**Sample request**

```bash
curl -X POST 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/RoutePoints' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  -d '{"routeid":3796}'
```

**Sample response**

```json
{
    "data": [
        {
            "latitude": "12.97751",
            "longitude": "77.57141",
            "responsecode": 200
        },
        {
            "latitude": "12.97749",
            "longitude": "77.57098",
            "responsecode": 200
        },
        {
            "latitude": "12.905319",
            "longitude": "77.543217",
            "responsecode": 200
        },
        {
            "latitude": "12.905319",
            "longitude": "77.543217",
            "responsecode": 200
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 560,
    "responsecode": 200
}
```


In [ ]:
#| export
def fetch_route_points(route_id: str, sleep_duration: float = 0.1):
    """Fetches geo-locations for a given BMTC `route_id` from the API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/RoutePoints"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    payload = {"routeid": int(route_id)}
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [ ]:
#| hide
#| eval: false

directory = data_directory / 'raw' / 'route_points' / str(int(datetime.datetime.now().timestamp()))
directory.mkdir(exist_ok=True, parents=True)

for index, row in tqdm(df_routes.iterrows(), total = df_routes.shape[0], desc = 'Fetching route points'):
    trip_details = fetch_route_points(route_id = row['route_id']) 
    with open(directory / f"{row['route_id']}.json", "w") as f:
        json.dump(trip_details, f, indent = 4)

In [ ]:
#| export
def get_route_id(filepath: Path):
    """The filepath has `route_id` at the end."""
    return filepath.name.split('.')[0]

In [ ]:
#| export
def convert_route_to_geojson(route, properties):
    """Converts route into a geojson Feature."""
    if route is None:
        return None
    coordinates = []
    for row in route['data']:
        # longitude is first, latitude comes next.
        coordinates.append([round(float(row['longitude']), 6), round(float(row['latitude']), 6)])
    geometry = geojson.LineString(coordinates)
    feature = geojson.Feature(geometry = geometry, properties = properties)
    return feature

In [ ]:
#| hide
#| eval: false

routes = []
directory = get_latest_directory(data_directory / 'raw' / 'route_points')
for filepath in tqdm(directory.ls(), total = directory.ls().__len__(), desc = 'Converting route points to geojson'):
    route = read_file(filepath)
    if route is None:
        continue
    properties = {'route_id': get_route_id(filepath)}
    routes.append(convert_route_to_geojson(route, properties))


# Write to a file as a geojson FeatureCollection.
fc = geojson.FeatureCollection(features = routes)
with open(data_directory / 'cleaned' / 'route_points.geojson', 'w') as f:
    json.dump(fc, f)

# Vehicles

> Returns parent ID and registration number of vehicles which match the vehicle number passed in the argument. Ex: `0007` returns `KA53F0007` and `KA57F0007`.

**Sample request**

```bash
curl 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/ListVehicles' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  --data-raw '{"vehicleRegNo":"0007"}'

```

**Sample response**

```json
{
    "data": [
        {
            "vehicleid": 19100,
            "vehicleregno": "KA53F0007",
            "responsecode": 200
        },
        {
            "vehicleid": 18830,
            "vehicleregno": "KA57F0007",
            "responsecode": 200
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 2,
    "responsecode": 200
}
```


In [ ]:
#| export
def fetch_vehicles(pattern: str = "", sleep_duration: float = 0.1):
    """Fetch vehicles matching the given registration number pattern from BMTC API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/ListVehicles"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if pattern != '':
        payload = {"vehicleRegNo": pattern}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        vehicles = []
        characters = string.digits + string.ascii_lowercase
        for pattern in tqdm(characters, desc = 'Fetching vehicles'):
            vehicles += fetch_vehicles(pattern)
        return vehicles

In [ ]:
#| hide
#| eval: false
vehicles = fetch_vehicles(pattern='0123')
print(json.dumps(vehicles, indent=4))

In [ ]:
#| export
def process_vehicles(vehicles):
    """Process and clean vehicle data, returning a DataFrame with `vehicle_id` and `registration_number`."""
    df_vehicles = pd.DataFrame(vehicles)
    df_vehicles = df_vehicles.drop_duplicates(subset=["vehicleregno"], keep="first")

    df_vehicles.rename(columns = {'vehicleid': 'vehicle_id', 'vehicleregno': 'registration_number'}, inplace=True)
    df_vehicles = df_vehicles.sort_values(by='vehicle_id').reset_index(drop=True)
    columns = ['vehicle_id', 'registration_number']
    return df_vehicles[columns]

In [ ]:
#| hide
#| eval: false
df_vehicles = process_vehicles(vehicles)
df_vehicles

# Trip details

> Takes a `vehicle_id` as input and returns route details of the current trip along with the live location of the vehicle.

**Sample request**

```bash
curl 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/VehicleTripDetails_v2' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  --data-raw '{"vehicleId":21670}'
```

**Sample response**

```json
{
    "RouteDetails": [
        {
            "rowid": 1,
            "tripid": 68043555,
            "routeno": "210-N",
            "routename": "KBS-UTH",
            "busno": "KA57F0614",
            "tripstatus": "Running",
            "tripstatusid": "1",
            "sourcestation": "Kempegowda Bus Station",
            "destinationstation": "Uttarahalli Bus Stand",
            "servicetype": "Non AC/Ordinary",
            "webservicetype": "Non-AC",
            "servicetypeid": 72,
            "lastupdatedat": "17-08-2025 12:33:18",
            "stationname": "Uttarahalli Bus Stand",
            "stationid": 22569,
            "actual_arrivaltime": null,
            "etastatus": "12:41",
            "etastatusmapview": "12:41",
            "latitude": 12.90535,
            "longitude": 77.54327,
            "currentstop": "",
            "laststop": "Gowdanapalya (Towards Uttarahalli)",
            "weblaststop": "Gowdanapalya",
            "nextstop": "Chikkallasandra Aralimara (Towards Uttarahalli)",
            "currlatitude": 12.911503,
            "currlongitude": 77.555923,
            "sch_arrivaltime": "12:48",
            "sch_departuretime": "12:48",
            "eta": "12:41",
            "actual_arrivaltime1": null,
            "actual_departudetime": null,
            "tripstarttime": "11:50",
            "tripendtime": "12:55",
            "routeid": 3796,
            "vehicleid": 21670,
            "responsecode": 200,
            "lastreceiveddatetimeflag": 1,
            "srno": 1584405201,
            "tripposition": 1,
            "stopstatus": 1,
            "stopstatus_distance": 1.53,
            "lastetaupdated": "2025-08-17T12:41:00",
            "minstopstatus_distance": 0.38
        }
    ],
    "LiveLocation": [
        {
            "latitude": 12.911503,
            "longitude": 77.555923,
            "location": "Gowdanapalya (Towards Kadirenahalli)",
            "lastrefreshon": "17-08-2025 12:33:18",
            "nextstop": "Chikkallasandra Aralimara (Towards Uttarahalli)",
            "previousstop": "Prarthana School (Towards Uttarahalli)",
            "vehicleid": 21670,
            "vehiclenumber": "KA57F0614",
            "routeno": "210-N",
            "servicetypeid": 72,
            "servicetype": "Non AC/Ordinary",
            "heading": 241.00,
            "responsecode": 200,
            "trip_status": 1,
            "lastreceiveddatetimeflag": 1
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 29,
    "responsecode": 200
}
```


In [ ]:
# | export
def fetch_trip_details(vehicle_id: int, sleep_duration: float = 0.1):
    """Fetch trip details for a given vehicle ID from the BMTC API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/VehicleTripDetails_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
    }

    payload = json.dumps({"vehicleId": int(vehicle_id)})
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        print("Response text:", getattr(e.response, "text", None))
        return None

In [ ]:
#| hide
#| eval: false

directory = data_directory / 'raw' / 'trip_details' / str(int(datetime.datetime.now().timestamp()))
directory.mkdir(exist_ok=True, parents=True)

for index, row in tqdm(df_vehicles.iterrows(), total = df_vehicles.shape[0], desc = 'Fetching trip details'):
    trip_details = fetch_trip_details(vehicle_id = row['vehicle_id']) 
    with open(directory / f"{row['vehicle_id']}.json", "w") as f:
        json.dump(trip_details, f, indent = 4)

**Issue 1. Vehicle is assigned to more than one route**

Vehicles on some occassions are assigned to more than one route. Ex: Vehicle `KA57F5808` is assigned to two routes, `routeno = 210-NA` and `routeno = D33-PPLO`.

```json
"LiveLocation": [
    {
        "latitude": 12.909809,
        "longitude": 77.536422,
        "location": "Depot-33 Poornapragna layout (Towards Depot-33 (Poornapragna layout))",
        "lastrefreshon": "17-09-2025 23:19:34",
        "nextstop": null,
        "previousstop": "Arehalli (Towards Kengeri)",
        "vehicleid": 27211,
        "vehiclenumber": "KA57F5808",
        "routeno": "210-NA",
        "servicetypeid": 72,
        "servicetype": "Non AC/Ordinary",
        "heading": 210.18,
        "responsecode": 200,
        "trip_status": 1,
        "lastreceiveddatetimeflag": 1
    },
    {
        "latitude": 12.909809,
        "longitude": 77.536422,
        "location": "Depot-33 Poornapragna layout (Towards Depot-33 (Poornapragna layout))",
        "lastrefreshon": "17-09-2025 23:19:34",
        "nextstop": null,
        "previousstop": "Arehalli (Towards Kengeri)",
        "vehicleid": 27211,
        "vehiclenumber": "KA57F5808",
        "routeno": "D33-PPLO",
        "servicetypeid": 72,
        "servicetype": "Non AC/Ordinary",
        "heading": 210.18,
        "responsecode": 200,
        "trip_status": 1,
        "lastreceiveddatetimeflag": 1
    }
]
```

**Issue 2. Live location is missing**

Live location was empty for about `2.5%` of vehicles (`185` out of `7,247` vehicles) when run on `2025-09-17`. Ex: Vehicle ID  `28622` with registration number `KA01AR4181`.

```json
{
    "RouteDetails": [],
    "LiveLocation": [],
    "Message": "No Records Found",
    "Issuccess": true,
    "exception": null,
    "RowCount": 0,
    "responsecode": 200
}
```

In [ ]:
# | export
def extract_live_location(trip_detail):
    """Extract live location from trip detail."""
    try:
        locations = trip_detail['LiveLocation']
    except TypeError as e:
        # When trip_detail is None.
        locations = []

    # When there are more than one live locations, it's mostly because of the vehicle assigned to more than one route at a time.
    # We could use the route details and live location to determine which is the right route that the vehicle is running on.
    return locations

In [ ]:
# | export
def extract_live_locations(directory: Path):
    """Extract live location for all trip details in a directory."""
    live_locations = []
    for filepath in tqdm(directory.ls(), total = directory.ls().__len__(), desc = 'Extracting live locations'):
        with open(filepath) as f:
            trip_detail = json.load(f)
            
            # Extract live locations.
            live_location = extract_live_location(trip_detail)
            live_locations += live_location
    live_locations = pd.DataFrame(live_locations)
    return live_locations

In [ ]:
#| hide
#| eval: false

directory = get_latest_directory(data_directory / 'raw' / 'trip_details')
df_live_locations = extract_live_locations(directory)

df_live_locations.to_csv(data_directory / 'cleaned' / 'live_locations.csv', index=False)
print(df_live_locations.shape)
df_live_locations.head(3)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()